In [1]:
import DL_SampleGeneration

import time
import numpy as np
from PIL import Image
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle

from ipywidgets import Layout, interact, IntSlider, FloatSlider

gen = DL_SampleGeneration.SampleGenerator()

exD = 5000
devD = 2500

exPT_cnt = 1000
devPT_cnt = 999

sample_cnt = 1000

exIntensity = 1.0
devIntensity = 0.7


Sparse_Subpixel_Convolution module initialized!
Sample width:  480


In [2]:
start = time.time()
samples, particle_positions, diffusion_coefs, pt_cnts, particles_in_sight_cnt, avg_diffusions = gen.GenSamples(exD, devD, exPT_cnt, devPT_cnt, sample_cnt, exIntensity, devIntensity, gen.step_cnt-1)
print("Sample generation took: {:.3f} s".format(time.time() - start))

np.savez("data.npz", samples, particle_positions, diffusion_coefs, pt_cnts, particles_in_sight_cnt, avg_diffusions)

Sample generation took: 13.457 s


In [3]:
data = np.load("data.npz")
samples                = data["arr_0"]
particle_positions     = data["arr_1"]
diffusion_coefs        = data["arr_2"]
pt_cnts                = data["arr_3"]
particles_in_sight_cnt = data["arr_4"]
avg_diffusions         = data["arr_5"]

sample_inds = np.insert(np.cumsum(pt_cnts), 0, 0)

def plot_step(step, sample_cur):
    
    begin = sample_inds[sample_cur]
    end = begin + pt_cnts[sample_cur]
    
    x = particle_positions[step, begin:end, 1]
    y = particle_positions[step, begin:end, 0]
    
    plt.subplot(1, 2, 1)
    plt.axis([-gen.FOV_edge, gen.sample_sz_px-gen.FOV_edge, -gen.FOV_edge, gen.sample_sz_px-gen.FOV_edge])
    plt.gca().add_patch(Rectangle((0, 0), gen.cam_fov_px, gen.cam_fov_px, edgecolor = 'red', fill=False))
    plt.gca().set_aspect('equal', adjustable='box')
    plt.scatter(x,y)
    
    imgs = samples[sample_cur]
    plt.subplot(1, 2, 2)
    plt.imshow(imgs[step, :, :])
    plt.axis([-0.5, gen.cam_fov_px-0.5, -0.5, gen.cam_fov_px-0.5])
    plt.scatter(x,y, c="red")
    plt.show()
    
    print(particles_in_sight_cnt[sample_cur], avg_diffusions[sample_cur])
#     print(imgs[step, :, :].min(), imgs[step, :, :].max())
    


    
interact(plot_step, step=IntSlider(min=0, max=gen.step_cnt - 1, layout=Layout(width='100%')), sample_cur=IntSlider(min=0, max=sample_cnt - 1, layout=Layout(width='100%')));

(1000, 32, 32, 32)


interactive(children=(IntSlider(value=0, description='step', layout=Layout(width='100%'), max=31), IntSlider(v…